In [12]:
import os
import sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob
import csv

In [13]:
# Projekt-Setup
project_root = os.getcwd()
src_dir = os.path.join(project_root, "src")
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)


In [14]:
# Custom-Module importieren
from otsu_global import otsu_threshold, binarize
from gray_hist import compute_gray_histogram
from Dice_Score import dice_score

In [15]:
# Alle Bilder in allen img-Ordnern finden
img_paths = sorted(glob(os.path.join(project_root, "data-git", "**", "img", "*.*"), recursive=True))

# Ergebnisse sammeln
dice_results = []

for img_path in img_paths:
    rel_path = os.path.relpath(img_path, start=os.path.join(project_root, "data-git"))
    print(f"📂 Verarbeite: {rel_path}")

    # Grauwertbild laden
    img = Image.open(img_path).convert("L")
    img_array = np.array(img)

    # Histogramm & Schwellenwert
    hist, _ = compute_gray_histogram(img_array)
    p = hist / hist.sum()
    t = otsu_threshold(p)

    # Binarisieren
    pred = binarize(img_array, t)

    # 🔍 Ground-Truth suchen
    dataset = rel_path.split(os.sep)[0]
    img_name = os.path.basename(img_path)
    number = ''.join(filter(str.isdigit, img_name))  # z. B. t01.tif → 01
    gt_name = f"man_seg{number}.tif"
    gt_path = os.path.join(project_root, "data-git", dataset, "gt", gt_name)

    if os.path.exists(gt_path):
        gt = Image.open(gt_path).convert("L")
        gt_array = np.array(gt) > 0

        # Dice Score berechnen
        dice = dice_score(gt_array, pred)
        dice_results.append((rel_path, dice))
        print(f"✅ Dice Score: {dice:.4f}")
    else:
        print(f"⚠️  Keine Ground Truth gefunden für {img_name}")
        dice_results.append((rel_path, None))


📂 Verarbeite: N2DH-GOWT1\img\t01.tif
✅ Dice Score: 0.5705
📂 Verarbeite: N2DH-GOWT1\img\t21.tif
✅ Dice Score: 0.5738
📂 Verarbeite: N2DH-GOWT1\img\t31.tif
✅ Dice Score: 0.5680
📂 Verarbeite: N2DH-GOWT1\img\t39.tif
✅ Dice Score: 0.5830
📂 Verarbeite: N2DH-GOWT1\img\t52.tif
✅ Dice Score: 0.6502
📂 Verarbeite: N2DH-GOWT1\img\t72.tif
✅ Dice Score: 0.5882
📂 Verarbeite: N2DL-HeLa\img\t13.tif
✅ Dice Score: 0.0744
📂 Verarbeite: N2DL-HeLa\img\t52.tif
✅ Dice Score: 0.1236
📂 Verarbeite: N2DL-HeLa\img\t75.tif
✅ Dice Score: 0.4032
📂 Verarbeite: N2DL-HeLa\img\t79.tif
✅ Dice Score: 0.4113
📂 Verarbeite: NIH3T3\img\dna-0.png
⚠️  Keine Ground Truth gefunden für dna-0.png
📂 Verarbeite: NIH3T3\img\dna-1.png
⚠️  Keine Ground Truth gefunden für dna-1.png
📂 Verarbeite: NIH3T3\img\dna-26.png
⚠️  Keine Ground Truth gefunden für dna-26.png
📂 Verarbeite: NIH3T3\img\dna-27.png
⚠️  Keine Ground Truth gefunden für dna-27.png
📂 Verarbeite: NIH3T3\img\dna-28.png
⚠️  Keine Ground Truth gefunden für dna-28.png
📂 Verarbeite:

In [16]:
# 📄 CSV exportieren
csv_path = os.path.join(project_root, "output", "dice_scores.csv")
os.makedirs(os.path.dirname(csv_path), exist_ok=True)
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(["Bild", "DiceScore"])
    for path, score in dice_results:
        writer.writerow([path, score if score is not None else "NA"])

print(f"\n✅ Ergebnisse gespeichert unter: {csv_path}")


✅ Ergebnisse gespeichert unter: c:\Users\victo\SynologyDrive\Studium\molekulare Biotechnologie\Heidelberg\Bioinformatik\Projekt\Otsu Treshhold\Otsu_Thresholding_LocalWork\topic01_team03\output\dice_scores.csv
